# 乱数を使って保持器を作る

In [49]:
# `maturin develop --release` command will install the package in the current virtualenv
!cd ../ && maturin develop --release


⚠️  Warning: You're building a library without activating pyo3's `extension-module` feature. See https://pyo3.rs/v0.21.2/building_and_distribution.html#linking
🔗 Found pyo3 bindings
🐍 Found CPython 3.12 at /Users/gyakusu/Morphing/.env/bin/python
   Compiling morphing v0.1.0 (/Users/gyakusu/Morphing)
    Finished release [optimized] target(s) in 2.20sorphing                    
📦 Built wheel for CPython 3.12 to /var/folders/w_/pg1jg9jx7hgfsps7wys8dmfm0000gn/T/.tmpo1TmLi/morphing-0.1.0-cp312-cp312-macosx_11_0_arm64.whl
✏️  Setting installed package as editable
🛠 Installed morphing-0.1.0


In [50]:
from morphing.morphing import CageParameter, Brg


In [51]:
import numpy as np
import pandas as pd

In [52]:
np.random.seed(0)
params = np.random.randn(200, 10) / 3
params[params<-1] = -1
params[params>1] = 1
params[:2]

array([[ 0.58801745,  0.13338574,  0.32624599,  0.7469644 ,  0.62251933,
        -0.32575929,  0.31669614, -0.0504524 , -0.03440628,  0.13686617],
       [ 0.04801452,  0.48475784,  0.25367924,  0.04055834,  0.14795441,
         0.11122478,  0.49802636, -0.06838609,  0.1043559 , -0.28469858]])

In [53]:
params.max(), params.min()

(1.0, -1.0)

In [54]:
r0 = lambda dr: 2.35e-3 + dr
r1 = lambda dr: 2.85e-3 + dr

r0(0.25e-3), r1(-0.25e-3)

(0.0026, 0.0026)

In [55]:
r0s = r0(params[:, 0] * 0.2e-3)
r1s = r1(params[:, 1] * 0.2e-3)
drs = r1s - r0s

drs.min() * 1e3, drs.max() * 1e3, r0s.min() * 1e3, r1s.max() * 1e3


(0.27704138687222785,
 0.7375057607384937,
 2.1778135578020037,
 3.026195714734316)

In [56]:
h0 = lambda dh: 0.93e-3 + dh
h1 = lambda dh: 2.10e-3 + dh

h0s = h0(params[:, 2] * 0.2e-3)
h1s = h1(params[:, 3] * 0.2e-3)

h0s.max() * 1e3, h0s.min() * 1e3, h1s.max() * 1e3, h1s.min() * 1e3

(1.0958961076220752,
 0.7796290513731874,
 2.2839570076014386,
 1.9305840794874516)

In [57]:
bevel = lambda r: 0.1 * drs * r

bevels = bevel(params[:, 4]+2)

(bevels / drs).max(), (bevels / drs).min()

(0.30000000000000004, 0.11136092540010864)

In [58]:
pocket_r = lambda dr: 0.840e-3 + dr

pocket_rs = pocket_r(params[:, 5] * 0.040e-3)
pocket_rs.min() * 1e3, pocket_rs.max() * 1e3

(0.8022059399296706, 0.8721660490605834)

In [59]:
neck_x = lambda dx: 1.70e-3 + dx

neck_xs = neck_x(params[:, 6] * 20e-6)
neck_xs.min() * 1e3, neck_xs.max() * 1e3

(1.680035914265149, 1.7174704923067883)

In [60]:
neck_r = lambda dr: 1.20e-3 + dr

neck_rs = neck_r(params[:, 7] * 20e-6)
neck_rs.min() * 1e3, neck_rs.max() * 1e3


(1.1817354855520694, 1.2169348774717277)

In [61]:
neck_h = lambda dh: 2.45e-3 + dh

neck_hs = neck_h(params[:, 8] * 100e-6)
neck_hs.min() * 1e3, neck_hs.max() * 1e3


(2.3565932400536096, 2.5393523613589664)

In [62]:
neck_dh = lambda r: (neck_hs - h1s) * r * 0.1

neck_dhs = neck_dh(params[:, 9]+2)
neck_dhs.min() * 1e3, neck_dhs.max() * 1e3


(0.02409880898608236, 0.12664880375765164)

In [68]:
df = pd.DataFrame({
    'r0': r0s,
    'r1': r1s,
    'h0': h0s,
    'h1': h1s,
    'bevel': bevels,
    'pocket_r': pocket_rs,
    'neck_x': neck_xs,
    'neck_r': neck_rs,
    'neck_h': neck_hs,
    'neck_dh': neck_dhs
})

# df.to_csv('../data/random/random_params.csv', index=False)
df.to_csv('../data/CFD/random/random_params.csv', index=False)


In [36]:
cage = CageParameter(2.345e-3,2.850e-3,0.93e-3,2.10e-3,0.10e-3,0.825e-3,1.70e-3,1.20e-3,2.45e-3,0.152e-3)
brg  = Brg("../data/CFD/star_master4_rotated.vtu", "../data/CFD/index4.xml", cage)
brg_std = brg.std()


In [37]:
new_cage = lambda i: CageParameter(r0s[i], r1s[i], h0s[i], h1s[i], bevels[i], pocket_rs[i], neck_xs[i], neck_rs[i], neck_hs[i], neck_dhs[i])
brg.reload_cage(new_cage(10), 100, 100)


In [38]:
# # cargo run --release "data/Tetra_standard.vtu,data/standard_index.xml,data/Tetra_standard_Smoothed.vtu" "2.345e-3,2.850e-3,0.93e-3,2.10e-3,0.10e-3,0.825e-3,1.70e-3,1.20e-3,2.45e-3,0.152e-3" "1000,1000,5,5"

# command_args0 = lambda i: f"\"data/Tetra_7431.vtu,data/index_7431.xml,data/random/Tetra_7431_{i}.vtu\" "
# command_args0 = lambda i: f"\"data/FEM/FEM2.vtu,data/FEM/index2.xml,data/FEM/random/FEM_{i}.vtu\" "
# command_args0(0)


In [39]:
# command_args1 = lambda i: f'\"{r0s[i]},{r1s[i]},{h0s[i]},{h1s[i]},{bevels[i]},{pocket_rs[i]},{neck_xs[i]},{neck_rs[i]},{neck_hs[i]},{neck_dhs[i]}\" \"1000,1000,30,5\"'
# command_args1(10)

# # '../data/Tetra_linspace10.vtu 0.0024755433798037503 0.002760149395923837 0.0008453010001009512 0.002164626447210534 4.579193726579163e-05 0.0008256478737285498 0.0016972425401282683 0.0011950169679237283 0.002514098067549346 0.0001742818416754026 1000'

In [40]:
# import subprocess

# command = "cd ../ && cargo run --release "
# result = subprocess.run(command + command_args0(0) + command_args1(0), shell=True)


In [ ]:
for i in range(len(df)):
    
    brg.reload_cage(new_cage(i), 1000, 1000)
    
    # 最初のステップ
    for _ in range(10):
        if brg.flip_with_scale(brg_std, 0.1):
            break
    else:  # for-else構文: ループが正常に終了した場合（breakされなかった場合）に実行
        print(f"Failed at iter0 {i}")
        continue  # 次のイテレーションへスキップ

    # 二番目のステップ
    for _ in range(5):
        if brg.flip_sphire_with_scale(brg_std, 0.01):
            print(f"Success {i}")
            brg.write_vtk_from_base("../data/CFD/star_master4_rotated.vtu", f"../data/CFD/random/CFD4_{i}.vtu")
            break
    else:
        print(f"Failed at iter1 {i}")

In [69]:
import os

file_name = lambda i: f'../data/CFD/random/CFD4_{i}.vtu'

renamelist = []

j = 0
for i in range(len(df)):
    
    if os.path.exists(file_name(i)):
        renamelist.append(i)
        j = j + 1

renamelist = np.array(renamelist[:100])
renamelist.shape


(100,)

In [70]:
renamelist

array([  0,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
        15,  17,  22,  23,  25,  26,  27,  31,  32,  33,  34,  35,  36,
        37,  39,  40,  42,  43,  44,  45,  46,  47,  49,  50,  51,  52,
        54,  55,  56,  57,  59,  60,  62,  63,  64,  65,  66,  67,  69,
        70,  71,  72,  74,  75,  78,  79,  80,  81,  82,  83,  84,  86,
        87,  88,  89,  90,  91,  92,  93,  97,  98,  99, 100, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 112, 113, 114, 115, 117, 118,
       119, 120, 121, 122, 124, 126, 128, 129, 131])

In [71]:
df = pd.read_csv('../data/CFD/random/random_params.csv')
df = df.iloc[renamelist]
df.to_csv('../data/CFD/random/random_params100.csv', index=False)

In [72]:
for i in range(100):
    os.rename(file_name(renamelist[i]), f'../data/CFD/random/CFD4_{i}.vtu')

In [ ]:
# import os

# file_name = lambda i: f'../data/random/Tetra_7431_{i}.vtu'

# j = 0
# for i in range(100):
#     is_exist = os.path.exists(file_name(i))
    
#     if not is_exist:
#         continue
    
#     # change file name
#     os.rename(file_name(i), file_name(j))
#     j = j + 1

### 次は　`VTK2CGNS.ipynb` へ

[VTK2CGNS.ipynb](VTK2CGNS.ipynb)